In [ ]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
global df


In [ ]:
def api_runner():
    from requests import Request, Session
    from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
    import json
    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '5f2075b9-4319-450c-999f-f94f88b8505d',
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    df

    if not os.path.isfile(r'C:\Users\PC\Documents\Mestrado\Curso\Python\Automating Crypto\API.csv'):
       df.to_csv(r'C:\Users\PC\Documents\Mestrado\Curso\Python\Automating Crypto\API.csv', header='column_names')
    else:
       df.to_csv(r'C:\Users\PC\Documents\Mestrado\Curso\Python\Automating Crypto\API.csv', mode='a', header=False)

  

In [ ]:

from time import time
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner completed')
    sleep(60)
exit()

In [ ]:
df = pd.read_csv(r'C:\Users\PC\Documents\Mestrado\Curso\Python\Automating Crypto\API.csv')

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
df2=df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df2

In [ ]:
df3 = df2.stack()
df3

In [ ]:
df4 = df3.to_frame(name='values')
df4 = df4.reset_index()
df4

In [ ]:
df4 = df4.rename(columns={'level_1': 'percent_change'})
df4

In [ ]:
df4['percent_change'] = df4['percent_change'].replace(
    ['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
     'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
     'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'],
    ['1h', '24h', '7d', '30d', '60d', '90d']
)
df4

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df4, kind='point')

In [ ]:
df5 = df[['name','quote.USD.price','timestamp']]
df5 = df5.query("name == 'Bitcoin'")
df5

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp',y='quote.USD.price',data = df5)